In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import os

In [3]:
os.chdir('/Users/Sriram/Desktop/DePaul/Work - LIDC (Smart CAD)')

In [6]:
data = np.loadtxt('./MalignancyVals.csv', delimiter=',', skiprows=1, dtype='int64')

In [7]:
#Choosing a single label based on mode, in case of tie, choose the first occurence
newLabels = np.zeros(shape=[data.shape[0]])
for i in range(data.shape[0]):
    bins = np.bincount(data[i,:])
    newLabels[i] = np.argmax(bins)

In [12]:
#S Matrix and D Matrix
S = np.zeros(shape=[newLabels.shape[0],newLabels.shape[0]])
D = np.ones(shape=[newLabels.shape[0],newLabels.shape[0]])
for i in range(len(newLabels)):
    for j in range(i,len(newLabels)):
        if newLabels[i] == newLabels[j]:
            S[i,j] = 1
            D[i,j] = 0

In [16]:
S = pd.DataFrame(S); D = pd.DataFrame(D)
S.to_csv('./SMatrix'); D.to_csv('./DMatrix')

In [30]:
df = pd.read_csv('./LIDC_REU2015.csv', index_col=0)
df.iloc[:,]

In [49]:
list(df.columns)

['StudyInstanceUID',
 'SeriesInstanceUid',
 'RadiologistID',
 'noduleID',
 'FilePath',
 'imageZposition',
 'imageSOP_UID',
 'coords',
 'PixelSpacing',
 'PixelArea',
 'Area',
 'ConvexArea',
 'Perimeter',
 'ConvexPerimeter',
 'EquivDiameter',
 'MajorAxisLength',
 'MinorAxisLength',
 'Elongation',
 'Compactness',
 'Eccentricity',
 'Solidity',
 'Extent',
 'Circularity',
 'RadialDistanceSD',
 'SecondMoment',
 'Roughness',
 'MinIntensity',
 'MaxIntensity',
 'MeanIntensity',
 'SDIntensity',
 'MinIntensityBG',
 'MaxIntensityBG',
 'MeanIntensityBG',
 'SDIntensityBG',
 'IntensityDifference',
 'markov1',
 'markov2',
 'markov3',
 'markov4',
 'markov5',
 'gabormean_0_0',
 'gaborSD_0_0',
 'gabormean_0_1',
 'gaborSD_0_1',
 'gabormean_0_2',
 'gaborSD_0_2',
 'gabormean_1_0',
 'gaborSD_1_0',
 'gabormean_1_1',
 'gaborSD_1_1',
 'gabormean_1_2',
 'gaborSD_1_2',
 'gabormean_2_0',
 'gaborSD_2_0',
 'gabormean_2_1',
 'gaborSD_2_1',
 'gabormean_2_2',
 'gaborSD_2_2',
 'gabormean_3_0',
 'gaborSD_3_0',
 'gabormean